In [4]:
# ========================================
# 🚀 CONVERTIR YOLO11 (.pt) A ONNX
# Para usar en Android con Buildozer
# ========================================

# 📦 PASO 1: Instalar Ultralytics
print("📦 Instalando Ultralytics YOLO...")
!pip install ultralytics onnx onnxruntime -q

print("✅ Instalación completa!\n")

# 📂 PASO 2: Subir tu modelo best.pt
print("=" * 50)
print("📂 SUBE TU ARCHIVO best.pt")
print("=" * 50)
print("👉 Haz clic en el ícono de carpeta a la izquierda")
print("👉 Luego en el botón 'Upload' (subir)")
print("👉 Selecciona tu archivo best.pt")
print("\n⏳ Esperando archivo...\n")

from google.colab import files
import os

# Subir archivo
uploaded = files.upload()

# Buscar archivo .pt (cualquier nombre)
pt_files = [f for f in uploaded.keys() if f.endswith('.pt')]

if not pt_files:
    print("❌ ERROR: No se encontró ningún archivo .pt")
    print("📝 Asegúrate de subir un archivo con extensión .pt")
    model_file = None
else:
    model_file = pt_files[0]
    print(f"✅ Archivo {model_file} cargado correctamente!")

    # Renombrar a best.pt para facilitar el proceso
    if model_file != 'best.pt':
        os.rename(model_file, 'best.pt')
        print(f"📝 Renombrado a: best.pt\n")
        model_file = 'best.pt'

# 🔄 PASO 3: Convertir a ONNX
print("=" * 50)
print("🔄 CONVIRTIENDO A ONNX...")
print("=" * 50)

from ultralytics import YOLO

try:
    # Verificar que tenemos un archivo
    if model_file is None:
        raise Exception("No se cargó ningún archivo .pt")

    # Cargar modelo
    print("📥 Cargando modelo...")
    model = YOLO(model_file)

    # Mostrar información del modelo
    print(f"\n📊 Información del modelo:")
    print(f"   - Clases detectadas: {len(model.names)}")
    print(f"   - Nombres de clases: {list(model.names.values())}")

    # Exportar a ONNX
    print("\n🔧 Exportando a ONNX (esto puede tardar 1-2 minutos)...")
    model.export(
        format='onnx',           # Formato ONNX
        imgsz=640,               # Tamaño de imagen (ajusta si entrenaste con otro)
        simplify=True,           # Simplifica el modelo (mejor para móviles)
        dynamic=False,           # Tamaño fijo (mejor rendimiento)
        opset=12                 # Versión ONNX (compatible con OpenCV)
    )

    print("\n✅ ¡Conversión exitosa!")
    print("📄 Archivo generado: best.onnx")

    # Verificar tamaño
    size_mb = os.path.getsize('best.onnx') / (1024 * 1024)
    print(f"📏 Tamaño del archivo: {size_mb:.2f} MB")

    if size_mb > 50:
        print("⚠️  ADVERTENCIA: El modelo es grande (>50MB)")
        print("   Considera usar cuantización para reducir tamaño")

except Exception as e:
    print(f"❌ ERROR en la conversión: {e}")
    print("\n🔍 Posibles soluciones:")
    print("   1. Verifica que best.pt sea un modelo YOLO válido")
    print("   2. Asegúrate de tener la versión correcta de ultralytics")
    print("   3. Intenta reiniciar el runtime de Colab")

# 🧪 PASO 4: Verificar que funciona
print("\n" + "=" * 50)
print("🧪 PROBANDO EL MODELO ONNX...")
print("=" * 50)

try:
    import onnxruntime as ort
    import numpy as np

    # Cargar modelo ONNX
    session = ort.InferenceSession('best.onnx')

    # Obtener información de entrada/salida
    input_info = session.get_inputs()[0]
    output_info = session.get_outputs()[0]

    print(f"✅ Modelo ONNX válido!")
    print(f"\n📥 Entrada:")
    print(f"   - Nombre: {input_info.name}")
    print(f"   - Shape: {input_info.shape}")
    print(f"   - Tipo: {input_info.type}")

    print(f"\n📤 Salida:")
    print(f"   - Nombre: {output_info.name}")
    print(f"   - Shape: {output_info.shape}")
    print(f"   - Tipo: {output_info.type}")

    # Prueba rápida con imagen dummy
    print("\n🎯 Haciendo inferencia de prueba...")
    dummy_input = np.random.rand(1, 3, 640, 640).astype(np.float32)
    outputs = session.run(None, {input_info.name: dummy_input})

    print(f"✅ Inferencia exitosa!")
    print(f"   - Shape de salida: {outputs[0].shape}")

except Exception as e:
    print(f"⚠️  Advertencia al verificar: {e}")
    print("   El modelo debería funcionar igual en Android")

# 💾 PASO 5: Descargar archivo
print("\n" + "=" * 50)
print("💾 DESCARGANDO best.onnx...")
print("=" * 50)

try:
    files.download('best.onnx')
    print("✅ Descarga iniciada!")
    print("\n📋 PRÓXIMOS PASOS:")
    print("   1. Guarda best.onnx en la carpeta de tu proyecto")
    print("   2. Verifica que esté junto a main.py")
    print("   3. Compila con: buildozer android debug")
except:
    print("⚠️  Si no se descargó automáticamente:")
    print("   Haz clic derecho en best.onnx → Download")

# 📊 PASO 6: Información adicional con conversión inteligente
print("\n" + "=" * 50)
print("📊 INFORMACIÓN PARA main.py")
print("=" * 50)

def convertir_nombre_a_valor(nombre):
    """Convierte nombres en español a valores numéricos"""
    nombre_upper = nombre.upper().replace(" ", "-")

    # Diccionario de conversión español a números
    numeros_texto = {
        "CINCUENTA": 50,
        "CIEN": 100,
        "DOSCIENTOS": 200,
        "QUINIENTOS": 500,
        "MIL": 1000,
        "DOS-MIL": 2000,
        "CINCO-MIL": 5000,
        "DIEZ-MIL": 10000,
        "VEINTE-MIL": 20000,
        "CINCUENTA-MIL": 50000,
        "CIEN-MIL": 100000,
    }

    # Buscar coincidencias exactas primero
    if nombre_upper in numeros_texto:
        return numeros_texto[nombre_upper]

    # Intentar extraer número directo
    import re
    match = re.search(r'^\d+$', nombre)
    if match:
        return int(match.group())

    # Buscar patrones compuestos (ej: "CINCO-MIL")
    for key, value in numeros_texto.items():
        if key in nombre_upper:
            return value

    # Si no se encuentra nada, retornar 0
    return 0

# Generar código para copiar
print("\n📝 Copia estos nombres de clases en tu main.py:")
print("=" * 50)

if 'model' in locals():
    class_names = list(model.names.values())
    print("CLASS_NAMES = [")
    for i, name in enumerate(class_names):
        print(f'    "{name}",  # Índice {i}')
    print("]")

    print("\n" + "=" * 50)
    print("💡 VALORES automáticos detectados:")
    print("=" * 50)
    print("VALORES = {")
    for name in class_names:
        valor = convertir_nombre_a_valor(name)
        if valor > 0:
            print(f'    "{name}": {valor},')
        else:
            print(f'    "{name}": 0,  # ⚠️ NO SE PUDO DETECTAR - AJUSTA MANUALMENTE')
    print("}")

    # Verificación de valores
    print("\n" + "=" * 50)
    print("🔍 VERIFICACIÓN DE VALORES:")
    print("=" * 50)
    for name in class_names:
        valor = convertir_nombre_a_valor(name)
        if valor == 0:
            print(f"⚠️  '{name}' → No se detectó valor automático")
        else:
            print(f"✅ '{name}' → ${valor:,}")

print("\n" + "=" * 50)
print("🎉 ¡PROCESO COMPLETADO!")
print("=" * 50)
print("\n✅ Tienes best.onnx listo para Android")
print("📱 Ahora compila tu app con buildozer")
print("🚀 ¡Buena suerte!")
print("=" * 50)

📦 Instalando Ultralytics YOLO...
✅ Instalación completa!

📂 SUBE TU ARCHIVO best.pt
👉 Haz clic en el ícono de carpeta a la izquierda
👉 Luego en el botón 'Upload' (subir)
👉 Selecciona tu archivo best.pt

⏳ Esperando archivo...



Saving v2best.pt to v2best.pt
✅ Archivo v2best.pt cargado correctamente!
📝 Renombrado a: best.pt

🔄 CONVIRTIENDO A ONNX...
📥 Cargando modelo...

📊 Información del modelo:
   - Clases detectadas: 11
   - Nombres de clases: ['100', '1000', '10000', '100000', '200', '2000', '20000', '50', '500', '5000', '50000']

🔧 Exportando a ONNX (esto puede tardar 1-2 minutos)...
Ultralytics 8.3.221 🚀 Python-3.12.12 torch-2.8.0+cu126 CPU (AMD EPYC 7B12)
YOLO11s summary (fused): 100 layers, 9,417,057 parameters, 0 gradients, 21.3 GFLOPs

PyTorch: starting from 'best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 15, 8400) (18.3 MB)

ONNX: starting export with onnx 1.19.1 opset 12...
ONNX: slimming with onnxslim 0.1.72...
ONNX: export success ✅ 2.2s, saved as 'best.onnx' (36.2 MB)

Export complete (3.3s)
Results saved to /content
Predict:         yolo predict task=detect model=best.onnx imgsz=640  
Validate:        yolo val task=detect model=best.onnx imgsz=640 data=./data/data.yaml 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Descarga iniciada!

📋 PRÓXIMOS PASOS:
   1. Guarda best.onnx en la carpeta de tu proyecto
   2. Verifica que esté junto a main.py
   3. Compila con: buildozer android debug

📊 INFORMACIÓN PARA main.py

📝 Copia estos nombres de clases en tu main.py:
CLASS_NAMES = [
    "100",  # Índice 0
    "1000",  # Índice 1
    "10000",  # Índice 2
    "100000",  # Índice 3
    "200",  # Índice 4
    "2000",  # Índice 5
    "20000",  # Índice 6
    "50",  # Índice 7
    "500",  # Índice 8
    "5000",  # Índice 9
    "50000",  # Índice 10
]

💡 VALORES automáticos detectados:
VALORES = {
    "100": 100,
    "1000": 1000,
    "10000": 10000,
    "100000": 100000,
    "200": 200,
    "2000": 2000,
    "20000": 20000,
    "50": 50,
    "500": 500,
    "5000": 5000,
    "50000": 50000,
}

🔍 VERIFICACIÓN DE VALORES:
✅ '100' → $100
✅ '1000' → $1,000
✅ '10000' → $10,000
✅ '100000' → $100,000
✅ '200' → $200
✅ '2000' → $2,000
✅ '20000' → $20,000
✅ '50' → $50
✅ '500' → $500
✅ '5000' → $5,000
✅ '50000' → $50,0